In [ ]:
!git clone https://github.com/loser666code/IMERSAO-AI---ALURA.git

Cloning into 'IMERSAO-AI---ALURA'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [4]:
%pip -q install google-genai

In [5]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [6]:
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"
from IPython.display import HTML, Markdown

In [7]:
!pip install -q google_adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
warnings.filterwarnings("ignore")

In [9]:
def chamarAgente(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
def agente_analista(crt):
      analista = Agent(
      name = "Analistadecaracteristicas",
      model = "gemini-2.0-flash",
      description="Esse agente recebe todas as caracteristica do usuario e então indica todos os atributos que o carro precisa ter",
      instruction="""Você é um especialista altamente experiente em veículos automotores, com foco específico na área de corretagem de automóveis.
       Sua tarefa é analisar detalhadamente a lista de características de uma entidade (que será fornecida) e,
       com base nessa análise, identificar e listar as necessidades e os requisitos para o carro ideal para essa entidade.
       Lembre-se que o público-alvo desta lista é outro especialista em automotores, portanto, a linguagem pode ser técnica e direta.
       A formatação de cada requisito e necessidade deve seguir o seguinte padrão:
       [Nome do Requisito/Necessidade]: [Justificativa detalhada e concisa, conectando a característica da entidade ao requisito do veículo.
       Explique o porquê dessa necessidade ser importante, considerando fatores como uso pretendido,
       perfil do usuário e quaisquer outras informações relevantes sobre a entidade.]
      """
     )
      entrada_analista = f"caracteristicas : {crt}"
      lista_necessidades = chamarAgente(analista, entrada_analista)
      return lista_necessidades

In [12]:
def agente_pesquisador(lista_necessidades):
      pesquisador = Agent(
      name = "Pesquisadordecarros",
      model = "gemini-2.0-flash",
      description="Esse agente recebe todas as caracteristica do usuario e então indica todos os atributos que o carro precisa ter",
      instruction="""
      Você é um especialista em veículos automotores, com profundo conhecimento em mecânica automotiva.
      Sua tarefa é analisar minuciosamente uma lista de necessidades para um carro (que será fornecida).
      Com base nessa análise, você deverá identificar um modelo de carro disponível no mercado brasileiro que melhor se compare e preencha esses requisitos,
      priorizando aqueles que atendem ao máximo de atributos listados.
      """
     )
      entrada_pesquisador = f"lista de necessidades: {lista_necessidades}"
      carro_ideal = chamarAgente(pesquisador, entrada_pesquisador)
      return carro_ideal

In [16]:
crt = input("quais as suas necessidades?")
listaNecessidades = agente_analista(crt)
display(to_markdown(listaNecessidades))
pesquisa = agente_pesquisador(listaNecessidades)
display(to_markdown(pesquisa))

quais as suas necessidades?Realiza entregas frequentes em áreas urbanas com ruas estreitas e tráfego intenso


> Aqui estão os requisitos e necessidades para o veículo ideal, com base nas características fornecidas:
> 
> *   **Manobrabilidade em Espaços Confinados:** Dada a frequência de entregas em áreas urbanas com ruas estreitas, o veículo precisa ter um raio de giro curto e dimensões compactas para facilitar manobras e estacionamento em espaços limitados.
> 
> *   **Durabilidade da Suspensão:** Considerando o uso urbano constante e a possibilidade de buracos e irregularidades nas vias, a suspensão do veículo deve ser robusta e durável, capaz de suportar o desgaste diário sem comprometer o conforto ou a capacidade de carga.
> 
> *   **Transmissão Ágil:** Para lidar com o tráfego intenso e as constantes paradas e partidas, o veículo necessita de uma transmissão responsiva (automática ou automatizada) que permita acelerações rápidas e retomadas eficientes, garantindo agilidade no trânsito.
> 
> *   **Visibilidade Aprimorada:** Em ambientes urbanos congestionados, a visibilidade é crucial. O veículo deve oferecer um campo de visão amplo e desobstruído, com espelhos retrovisores de grandes dimensões e, idealmente, recursos como câmera de ré e sensores de ponto cego para minimizar os riscos de acidentes.
> 
> *   **Economia de Combustível:** Considerando as frequentes entregas e o alto volume de quilometragem percorrida em áreas urbanas, a eficiência no consumo de combustível é fundamental para reduzir os custos operacionais. Priorizar veículos com motores de baixo consumo e tecnologias como start-stop.
> 
> *   **Capacidade de Carga Otimizada:** O veículo deve possuir um compartimento de carga com capacidade suficiente para acomodar as entregas típicas, com acesso fácil e rápido para otimizar o tempo de carregamento e descarregamento.
> 
> *   **Confiabilidade Mecânica:** A confiabilidade é crucial para evitar interrupções nas entregas. O veículo deve ser de uma marca com boa reputação em termos de durabilidade e baixo índice de manutenção, minimizando o tempo de inatividade por problemas mecânicos.
> 
> *   **Sistemas de Segurança Ativos:** Considerando o alto risco de acidentes em áreas urbanas movimentadas, o veículo deve ser equipado com sistemas de segurança ativos, como freios ABS, controle de estabilidade (ESP) e assistente de partida em rampa (HSA), para auxiliar o condutor em situações de emergência e reduzir o risco de colisões.


> Com base na sua lista detalhada de necessidades, o veículo ideal para suas operações de entrega em áreas urbanas no Brasil deve combinar agilidade, durabilidade, economia e segurança. Considerando esses fatores, o modelo que melhor se encaixa nas suas necessidades é o **Fiat Fiorino**.
> 
> **Justificativas:**
> 
> *   **Manobrabilidade:** O Fiat Fiorino é conhecido por suas dimensões compactas e excelente raio de giro, facilitando manobras em ruas estreitas e estacionamento em espaços limitados, típicos de áreas urbanas.
> *   **Durabilidade da Suspensão:** Projetado para uso comercial, o Fiorino possui uma suspensão robusta capaz de suportar as condições adversas das vias urbanas, incluindo buracos e irregularidades, sem comprometer a capacidade de carga.
> *   **Transmissão Ágil:** Embora o Fiorino não ofereça transmissão automática, sua transmissão manual bem ajustada proporciona trocas de marcha suaves e eficientes, permitindo acelerações rápidas e retomadas ágeis no trânsito urbano.
> *   **Visibilidade Aprimorada:** O Fiorino oferece boa visibilidade frontal e lateral, com espelhos retrovisores amplos. A instalação de acessórios como câmera de ré pode ser considerada para melhorar ainda mais a visibilidade em manobras.
> *   **Economia de Combustível:** Equipado com um motor 1.4 flex, o Fiorino oferece bom consumo de combustível, especialmente em uso urbano, contribuindo para a redução dos custos operacionais.
> *   **Capacidade de Carga Otimizada:** O compartimento de carga do Fiorino é projetado para acomodar um volume razoável de entregas, com acesso fácil e rápido pela porta traseira e pelas portas laterais corrediças, otimizando o tempo de carregamento e descarregamento.
> *   **Confiabilidade Mecânica:** A Fiat possui uma boa reputação em termos de durabilidade e baixo índice de manutenção, tornando o Fiorino uma escolha confiável para uso comercial.
> *   **Sistemas de Segurança Ativos:** O Fiorino é equipado com freios ABS e controle de estabilidade (ESP), garantindo a segurança do condutor em situações de emergência.
> 
> **Considerações Adicionais:**
> 
> *   **Custo-Benefício:** O Fiat Fiorino oferece um bom custo-benefício, combinando um preço acessível com baixo custo de manutenção e boa disponibilidade de peças de reposição.
> *   **Adaptações:** O Fiorino pode ser facilmente adaptado para atender às necessidades específicas da sua operação, como a instalação de prateleiras ou divisórias no compartimento de carga.
> 
> Embora outros modelos possam atender a alguns dos seus requisitos, o Fiat Fiorino se destaca como a opção mais equilibrada e adequada para suas necessidades de entrega em áreas urbanas no Brasil.


'\n'